In [3]:
import csv
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

In [4]:
with open('checkins.dat') as dat_file, open('checkins.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)ых 

    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)

In [5]:
data = pd.read_csv('checkins.csv')
data = data.iloc[:100000, :].loc[:, ['latitude', 'longitude']]
data.shape

(100000, 2)

In [6]:
model = MeanShift(bandwidth=0.1)
model.fit(data)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
          n_jobs=None, seeds=None)

In [18]:
labels = model.labels_
clusters_centers = model.cluster_centers_

labels_index, labels_counts = np.unique(labels, return_counts = True)
print(labels_index, labels_counts)

[   0    1    2 ... 3228 3229 3230] [12506  4692  3994 ...     1     1     1]


In [20]:
labels_unique_more = list()
for index, count in enumerate(labels_counts):
    if count > 15:
        labels_unique_more.append(labels[index])
clusters_centers_more = np.empty( (len(labels_unique_more), 2) )
for index, value in enumerate(labels_unique_more):
    clusters_centers_more[index] = clusters_centers[value]

In [10]:
offices = pd.read_csv('centers.txt', skipinitialspace=True)
offices_for_dist = offices.loc[:, ['latitude', 'longitude']]
offices_for_dist

,latitude,longitude
0,33.751277,-118.188740
1,25.867736,-80.324116
2,51.503016,-0.075479
3,52.378894,4.885084
4,39.366487,117.036146
5,-33.868457,151.205134


In [11]:
def distance(x, y):
    return np.sqrt(np.sum((x-y)**2))

In [45]:
distances_from_nearest_office = np.empty( clusters_centers_more.shape[0] )
for i, item in enumerate(clusters_centers_more):
    min_dist = distance(item, offices_for_dist.loc[0])
    for j in range(len(offices)):
        dist = distance(item, offices_for_dist.loc[j])
        if dist < min_dist:
            min_dist = dist
    distances_from_nearest_office[i] = min_dist

In [71]:
dist_sorted_indices = np.argsort(distances_from_nearest_office)
distances_from_nearest_office_sorted = distances_from_nearest_office[dist_sorted_indices]
clusters_centers_more_sorted = clusters_centers_more[dist_sorted_indices]

In [72]:
print('Minimal distance: %f' % distances_from_nearest_office_sorted[0])
print('Coordinates of closest centroid: %s' % str(clusters_centers_more_sorted[0]))

Minimal distance: 0.007835
Coordinates of closest centroid: [-33.86063043 151.20477593]


In [73]:
answer = str(cluster_centers_more_sorted[0, 0]) + ' ' + str(cluster_centers_more_sorted[0, 1])
with open('answer.txt', 'w') as fout:
    fout.write(answer)